# This section uses Magpie to calculate surface brightness profile

This is equivalently what the previous section does, but magpie is used to calculate surface brightness profile for each Ha Hb line maps and weights, averaged out over the polar angle phi, for each given pixel distance.

what does the package [magpie](https://github.com/knaidoo29/magpie/tree/master) do:

It transforms grid in Cartesian coordinates into (r,phi) polar coordinates, while preserving the size of the surface area of each pixel. Therefore one can calculate surface brightness of a certain ring area by summing over phi of a given radius range. This method is used in [Matharu 2023](https://iopscience.iop.org/article/10.3847/2041-8213/acd1db/pdf) and [Matharu 2024](https://arxiv.org/pdf/2404.17629).

Update: Now this script also calculates dust attenuation assuming [Calzetti et al. (2000)](https://ui.adsabs.harvard.edu/abs/2000ApJ...533..682C/abstract)

In [1]:
import  magpie              as     magpie

### calculate radial profile

In [2]:
import  numpy               as     np
from    scripts.tools       import *
from    astropy.table       import Table
from    astropy.io          import fits
from    astropy.cosmology   import Planck18
import  astropy.units       as     u
import  matplotlib.pyplot   as     plt
import  matplotlib.colors   as     colors  
from    matplotlib          import use
from    tqdm                import tqdm
from    concurrent.futures  import ThreadPoolExecutor, as_completed
import  sys, os
from    IPython.display     import clear_output
from    astropy.wcs         import WCS

obj_lis = Table.read('obj_lis_selected.fits')


In [5]:
def K_lambda(line='Ha'):
    #calzetti_attenuation
    """
    Calculate the dust attenuation value k(lambda) from the Calzetti et al. (2000) attenuation curve.
    Parameters:
        wavelength_nm (float): Wavelength in nanometers (nm).
    """
    wavelength_um =  (0.6563 if line == 'Ha' else 0.4861)
    if 0.12 <= wavelength_um <= 0.63:
        # Formula for the UV to optical wavelength range
        k_lambda = 2.659 * (-1.857 + 1.040 / wavelength_um) + 4.05
    elif wavelength_um > 0.63:
        # Formula for the near-infrared wavelength range
        k_lambda = 2.659 * (-2.156 + 1.509 / wavelength_um - 0.198 / (wavelength_um ** 2) + 0.011 / (wavelength_um ** 3)) + 4.05
    return k_lambda


import magpie.montecarlo
#use the montecarlo module to remap the data to polar coordinates
def spatial_remap(map,pixel_length,r_bins=50):
    b2r = magpie.montecarlo.Box2Ring()
    b2r.setup_box(-24.5*pixel_length, 24.5*pixel_length, 50,
                  -24.5*pixel_length, 24.5*pixel_length, 50)
    b2r.setup_polar_lin(0., 24.5*pixel_length, r_bins, 180, center=[0., 0.])
    b2r.get_weights() 
    return np.linspace(b2r.redges[0], b2r.redges[-1],r_bins), b2r.remap(map)


#this function will return the radial profile of a given object
#function: calls the magpie function spatial_remap to remap the data to polar coordinates
def radial_profile(obj,linemap,weight,seg,pixel_length):
        linemap = linemap.data; weight = weight.data
        #convert the seg map to a binary map, for further conversion to weight map
        seg = np.where(seg==obj['ID'],1,0)
        seg = spatial_remap(seg,pixel_length)[1]
        seg_weight = seg/np.sum(seg,axis=0)

        r, linemap_r,   = spatial_remap(np.where(linemap>0,linemap,0),pixel_length)
        linemap_r_wht = spatial_remap(np.where(linemap>0,weight,0),pixel_length)[1]


        map_r   = np.average(linemap_r,weights=seg_weight,axis=0)
        map_r_err = 1/np.sum(linemap_r_wht*seg,axis=0)**0.5
        map_r_std = np.average((linemap_r-map_r)**2,weights = seg_weight,axis=0)**0.5
        map_r_err = (map_r_err**2 + map_r_std**2)**0.5
        return r, map_r, map_r_err 

#this function will generate the             print(r, ha_r, ha_r_err, hb_r, hb_r_err, balmer_r, balmer_r_err)
#radial table for a given object
def gen_radial_table(obj,
                     LINE_HA='LINE_HA',      LINE_HB='LINE_HB_CONV',
                     LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV'):
    #try:
    path = f"data_extracted/{file_name(obj,prefix='extracted')}"
    with fits.open(path,mode='update') as hdu:
        if find_data('SEG_MOD',hdu) != None:
            seg_map = find_data('SEG_MOD',hdu)[1].data
        else:
            seg_map = find_data('SEG',hdu)[1].data

        #extract the radial profile surface brightness
        r, ha_r, ha_r_err = radial_profile(obj,
                                        linemap      = find_data(LINE_HA,hdu)[1],
                                        weight       = find_data(LINEWHT_HA,hdu)[1],
                                        seg          = seg_map,
                                        pixel_length = obj['pixel_length'])
        
        r, hb_r, hb_r_err = radial_profile(obj,
                                        linemap      = find_data(LINE_HB,hdu)[1],
                                        weight       = find_data(LINEWHT_HB,hdu)[1],
                                        seg          = seg_map,
                                        pixel_length = obj['pixel_length'])

        #calculate the balmer decrement
        balmer_r     = ha_r/hb_r
        balmer_r_err = ((ha_r_err/hb_r)**2 + (hb_r_err**2 * (ha_r/hb_r**2)**2))**0.5
        
        #now calculate the extinction
        #color excess
        E_ba = 2.5*np.log10(balmer_r/2.86)
        #attenutation
        A_ba = E_ba/(K_lambda('Hb')-K_lambda('Ha')) * K_lambda('Ha')

        #columns for the radial table
        cols = [
            fits.Column(name='DISTANCE [kpc]',                       format='E', array=r),
            fits.Column(name='Ha_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=ha_r),
            fits.Column(name='Ha_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=ha_r_err),
            fits.Column(name='Hb_SURF_BRIGHT [1e-17 erg/s/cm2]',     format='E', array=hb_r),
            fits.Column(name='Hb_SURF_BRIGHT_err [1e-17 erg/s/cm2]', format='E', array=hb_r_err),
            fits.Column(name='BALMER_DECREM',                        format='E', array=balmer_r),
            fits.Column(name='BALMER_DECREM_ERR',                    format='E',array=balmer_r_err),
            fits.Column(name='E_BV',                                  format='E', array=E_ba),
            fits.Column(name='A_Ha',                                  format='E', array=A_ba)
        ]
        
        #choose the right name for saving the radial table
        if 'CONV' in LINE_HB:
            name_addon = '_CONV'
        else:
            name_addon = ''
        if 'BG' not in LINE_HA:
            name = f'RAD_PROFILE{name_addon}'
            new_table = fits.BinTableHDU.from_columns(cols, name=name)
        else:
            name = f'RAD_PROFILE{name_addon}_BG'
            new_table = fits.BinTableHDU.from_columns(cols, name=name)

        #save or update table
        save_update(new_table,hdu)
        hdu.flush()
        return f"{obj['subfield']}-{obj['ID']} processed"
    #except Exception as e:
    #        return f"! {obj['subfield']}-{obj['ID']} failed, error{e}"

def cat_process(obj_lis,
                LINE_HA,   LINE_HB,
                LINEWHT_HA, LINEWHT_HB,
                max_threads=1):
        print(f'start process,{LINE_HA},{LINE_HB}')
        results = []
        if max_threads > 1 :
            with ThreadPoolExecutor(max_threads) as executor:
                futures = {executor.submit(
                    gen_radial_table,
                    obj,LINE_HA,LINE_HB,LINEWHT_HA,LINEWHT_HB
                                            ): obj for obj in obj_lis}
                for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                    results.append(future.result())
            return results
        else:
            for obj in tqdm(obj_lis):
                results.append(gen_radial_table(obj,LINE_HA,LINE_HB,LINEWHT_HA,LINEWHT_HB))
            return results


def main():
    obj_lis = Table.read('obj_lis_selected.fits')
    
    results1 = cat_process(obj_lis,
                        LINE_HA='LINE_HA',LINE_HB='LINE_HB',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB',max_threads=1)
    errorcounting(results1)

    results3 = cat_process(obj_lis,
                        LINE_HA='LINE_HA',LINE_HB='LINE_HB_CONV',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV',max_threads=1)
    errorcounting(results3)

'''
    results2 = cat_process(obj_lis,
                        LINE_HA='LINE_HA_BG',LINE_HB='LINE_HB_BG',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB',max_threads=6)
    errorcounting(results2)
    
    results4 = cat_process(obj_lis,
                        LINE_HA='LINE_HA_BG',LINE_HB='LINE_HB_CONV_BG',
                        LINEWHT_HA='LINEWHT_HA',LINEWHT_HB='LINEWHT_HB_CONV',max_threads=6)
    errorcounting(results4)
'''
if __name__ == '__main__':
    main()


start process,LINE_HA,LINE_HB


  0%|          | 0/158 [00:00<?, ?it/s]/tmp/ipykernel_18044/3594900428.py:36: RuntimeWarning: invalid value encountered in divide
  seg_weight = seg/np.sum(seg,axis=0)
/tmp/ipykernel_18044/3594900428.py:43: RuntimeWarning: divide by zero encountered in divide
  map_r_err = 1/np.sum(linemap_r_wht*seg,axis=0)**0.5
  2%|▏         | 3/158 [01:41<1:26:43, 33.57s/it]/tmp/ipykernel_18044/3594900428.py:80: RuntimeWarning: divide by zero encountered in log10
  E_ba = 2.5*np.log10(balmer_r/2.86)
  6%|▌         | 9/158 [05:01<1:22:34, 33.25s/it]/tmp/ipykernel_18044/3594900428.py:75: RuntimeWarning: divide by zero encountered in divide
  balmer_r     = ha_r/hb_r
/tmp/ipykernel_18044/3594900428.py:76: RuntimeWarning: divide by zero encountered in divide
  balmer_r_err = ((ha_r_err/hb_r)**2 + (hb_r_err**2 * (ha_r/hb_r**2)**2))**0.5
  7%|▋         | 11/158 [06:07<1:21:20, 33.20s/it]/tmp/ipykernel_18044/3594900428.py:75: RuntimeWarning: invalid value encountered in divide
  balmer_r     = ha_r/hb_r
/t

total number of obj processed: 158
number of failed obj 0
start process,LINE_HA,LINE_HB_CONV


100%|██████████| 158/158 [1:33:30<00:00, 35.51s/it]

total number of obj processed: 158
number of failed obj 0


### plot radial profiles

In [3]:
%matplotlib inline

def plot_balmer_decrem(obj, plot, plot_var, crop_size=30):
        # Construct the path to the FITS file
        path = f"data_extracted/{file_name(obj, prefix='extracted')}"
        
        # Open the FITS file
        with fits.open(path) as hdu:
            # Determine the center of the image
            shape = hdu[3].data.shape[0]
            si = (shape - crop_size) // 2; 
            ei = si + crop_size

            #extract segmentation map
            if find_data('SEG_MOD',hdu) != None:
                seg = find_data('SEG_MOD',hdu)[1].data
            else:
                seg = find_data('SEG',hdu)[1].data

            #effective radius (in arcsec)
            r_eff = obj['re']
             
            # Create a figure with subplots
            fig, axes = plt.subplots(2, 3, figsize=(20, 10))
            axes = axes.flatten()




            # Loop through the specified image names and plot them
            #here i try to add segmentation map effect to the image
            for i, name in enumerate(['DSCI', 'LINE_HA', 'LINE_HB_CONV']):
                image = find_data(name, hdu)[1]
                titles = {
    "DSCI": image.header['EXTTYPE'],
    "LINE_HA": r"H$\alpha$",
    "LINE_HB_CONV": r"H$\beta$"
}
                data = image.data[si:ei, si:ei]
                # Extract the segmentation map for the same region
                seg_crop = seg[si:ei, si:ei]
                # Create a mask where the segmentation map matches the object ID
                mask = seg_crop == obj['ID']
                
                ax = axes[i]
                ax.tick_params(direction='in',which='both', top=True, right=True)

                # Plot the original data with plasma_r colormap
                im = ax.imshow(np.where(mask,data,np.nan), norm=colors.Normalize(vmin=0,vmax=data.max()), origin='lower', cmap='plasma_r')
                # Overlay the segmentation map with gray colormap and lower transparency
                ax.imshow(np.where(np.logical_not(mask),data,np.nan), norm=colors.Normalize(vmin=0,vmax=data.max()), origin='lower', cmap='Greys_r', alpha=0.5)
                # Plot a circle representing the effective radius
                circ = plt.Circle((crop_size/2-1, crop_size/2-1), r_eff / 0.1, color='blue', fill=False, linestyle='--',label='effective radius',linewidth=2)
                ax.add_patch(circ)

                ax.plot([3, 7], [4, 4])
                ax.text(5, 5, f"{round(obj['pixel_length'] * 4, 2)} kpc")
                ax.set_title(f'{titles[name]}')
                ax.set_xticklabels([]);ax.set_yticklabels([])
                fig.colorbar(im, ax=ax)
                ax.legend()



            # Plot the 2D Balmer decrement image
            name = '2D_BALMER'
            if True:
                image = find_data(name, hdu)[1]
                data = image.data[si:ei, si:ei]
                ax = axes[3]
                im = ax.imshow(np.where(mask,data,np.nan), norm=colors.Normalize(vmin=0, vmax=12), origin='lower', cmap='plasma_r')
                ax.imshow(np.where(np.logical_not(mask),data,np.nan), norm=colors.Normalize(vmin=0, vmax=12),origin='lower', cmap='Greys_r', alpha=0.5)
                circ = plt.Circle((crop_size/2-1, crop_size/2-1), r_eff / 0.1, color='blue', fill=False, linestyle='--',label='effective radius',linewidth=2)
                ax.plot([3, 7], [4, 4])
                ax.text(5, 5, f"{round(obj['pixel_length'] * 4, 2)} kpc")
                ax.set_title(r'$H\alpha / H\beta$')
                ax.tick_params(direction='in',which='both', top=True, right=True)
                ax.set_xticklabels([]);ax.set_yticklabels([])
                fig.colorbar(im, ax=ax)



            # Extract and plot the radial profiles
            r, ha_r, ha_r_err, hb_r, hb_r_err, balmer_r, balmer_r_err, E_BV, A_Ha = np.vstack(find_data(plot, hdu)[1].data).transpose()
            r, ha_r_var, ha_r_err_var, hb_r_var, hb_r_err_var, balmer_r_var, balmer_r_err_var, E_BV, A_Ha = np.vstack(find_data(plot_var, hdu)[1].data).transpose()


            if True:
                # Plot the Ha and Hb radial profiles
                ax = axes[4]
                #mask = 
                ax.errorbar(r, ha_r, yerr=ha_r_err, fmt='ro:', label=r'H$\alpha$')
                ax.errorbar(r, hb_r, yerr=hb_r_err, fmt='go:', label=r'H$\beta$')
                ax.errorbar(r, hb_r_var, yerr=hb_r_err_var, fmt='go:', label=r'H$\beta$, psf matched', alpha=0.4)


                #effective radius
                ax.axvspan(0, r_eff * obj['pixel_length'] / 0.1, color='grey', alpha=0.3)
                
                # Determine the plot limits
                arr = np.array((ha_r, hb_r))
                arr = arr[arr > 0]
                plot_min = np.nanmin(arr) * 0.05
                plot_max = np.nanmax(arr) * 100

                # Annotate the effective radius
                ax.annotate(
                    "",  # Only draw the arrow, no text
                    xy=(0, plot_min * 2),  # Right endpoint
                    xytext=(r_eff * obj['pixel_length'] / 0.1, plot_min * 2),  # Left endpoint
                    arrowprops=dict(arrowstyle="<->", lw=1.5, color="black"),  # Horizontal double arrow
                )
                ax.text(r_eff * obj['pixel_length'] / 0.1 / 2, plot_min * 2.1, 
                        r"$r_{eff}$", fontsize=12, color="black", ha="center",va='bottom')
                ax.set_xlabel('r [kpc]')
                ax.set_ylabel('flux [$10^{-17}$ erg/s/cm$^2$$]')
                ax.set_ylim(plot_min, plot_max)
                ax.set_yscale('log')
                ax.tick_params(direction='in',which='both', top=True, right=True)
                ax.legend()



            if True:
                # Plot the Balmer decrement radial profile
                ax = axes[5]
                mask = np.logical_and(np.logical_not(np.isnan(balmer_r)),balmer_r>0)
                #ax.errorbar(r[mask], balmer_r[mask], yerr=balmer_r_err[mask], fmt='ro:', label=f'balmer_decrem_{plot}')
                ax.errorbar(r[mask], balmer_r_var[mask], yerr=balmer_r_err_var[mask],color='grey', fmt='o:', label=r'$H\alpha / H\beta$')
                ax.legend(loc='upper left')
                ax.set_xlabel('distance [kpc]')
                ax.set_ylabel(r'$H\alpha / H\beta$')
                ax.set_ylim(-5, 15)
                # Annotate the effective radius
                plot_min = -4.5
                #effective radius
                ax.axvspan(0, r_eff * obj['pixel_length'] / 0.1, color='grey', alpha=0.3)
                ax.annotate(
                    "",  # Only draw the arrow, no text
                    xy=(0, plot_min),  # Right endpoint
                    xytext=(r_eff * obj['pixel_length'] / 0.1, plot_min),  # Left endpoint
                    arrowprops=dict(arrowstyle="<->", lw=1.5, color="black"),  # Horizontal double arrow
                )
                ax.text(r_eff * obj['pixel_length'] / 0.1 / 2, plot_min, 
                        r"$r_{eff}$", fontsize=12, color="black", ha="center",va='bottom')

                # Plot the A_Ha radial profile on a secondary y-axis
                ax2 = ax.twinx()
                l2 =ax2.errorbar(r[mask], A_Ha[mask], yerr=balmer_r_err[mask], color='darkblue', linestyle='-', fmt='o', label=r'Attenuation H$\alpha$')
                ax2.set_ylabel(r'$A_{H\alpha}$')
                ax2.legend()
                ax.tick_params(direction='in',which='both', top=True, right=False)
                ax2.tick_params(direction='in',right=True)

            # Save the plot
            save_path = f"radial_balmer_decrem/{plot}_vs_{plot_var}"
            save_path_sn_10 = f"sn_10/radial_balmer_decrem/{plot}_vs_{plot_var}"
            os.makedirs(save_path, exist_ok=True)
            os.makedirs(save_path_sn_10, exist_ok=True)

            plt.savefig(f"{save_path}/{obj['subfield']}-{obj['ID']}_{obj['tag']}_{obj['manual_select']}.png")
            if obj['sn_hb'] > 10:
                plt.savefig(f"{save_path_sn_10}/{obj['subfield']}-{obj['ID']}_{obj['tag']}_{obj['manual_select']}.png",dpi=300)
            plt.close('all')

            return f"{obj['subfield']}-{obj['ID']} saved"

def cat_process(obj_lis, plot='RA D_PROFILE', plot_var='RAD_PROFILE_BG', max_threads=1):
    print(f'\n start plot process{plot,plot_var}')
    results = []
    if max_threads > 1:
        # Use multithreading to process the objects in parallel
        with ThreadPoolExecutor(max_threads) as executor:
            futures = {executor.submit(plot_balmer_decrem, obj, plot=plot, plot_var=plot_var): obj for obj in obj_lis}
            for future in tqdm(as_completed(futures), total=len(obj_lis), desc="Processing"):
                results.append(future.result())
        return results
    else:
        # Process the objects sequentially
        for obj in tqdm(obj_lis):
            results.append(plot_balmer_decrem(obj, plot, plot_var))
        return results

def main():
    use('Agg')  # Use the 'Agg' backend for matplotlib
    obj_lis = Table.read('obj_lis_selected.fits') # Read the object list from a FITS file
    
    # Process the objects and plot the results
    results1 = cat_process(obj_lis, plot='RAD_PROFILE', plot_var='RAD_PROFILE_CONV', max_threads=1)
    errorcounting(results1)
    print(results1)

if __name__ == '__main__':
    main()


 start plot process('RAD_PROFILE', 'RAD_PROFILE_CONV')


  6%|▌         | 9/158 [00:10<02:21,  1.05it/s]/home/yang/anaconda3/envs/magpie/lib/python3.9/site-packages/matplotlib/axes/_axes.py:3749: RuntimeWarning: invalid value encountered in add
  low, high = dep + np.vstack([-(1 - lolims), 1 - uplims]) * err
  8%|▊         | 12/158 [00:13<02:31,  1.03s/it]/home/yang/anaconda3/envs/magpie/lib/python3.9/site-packages/matplotlib/axes/_axes.py:3749: RuntimeWarning: invalid value encountered in add
  low, high = dep + np.vstack([-(1 - lolims), 1 - uplims]) * err
 20%|██        | 32/158 [00:40<02:42,  1.29s/it]/home/yang/anaconda3/envs/magpie/lib/python3.9/site-packages/matplotlib/axes/_axes.py:3749: RuntimeWarning: invalid value encountered in add
  low, high = dep + np.vstack([-(1 - lolims), 1 - uplims]) * err
 32%|███▏      | 50/158 [01:04<02:19,  1.30s/it]/home/yang/anaconda3/envs/magpie/lib/python3.9/site-packages/matplotlib/axes/_axes.py:3749: RuntimeWarning: invalid value encountered in add
  low, high = dep + np.vstack([-(1 - lolims), 1 - 

total number of obj processed: 158
number of failed obj 0
['GN2-10512 saved', 'GN2-11228 saved', 'GN7-11839 saved', 'GN7-11883 saved', 'GN7-12769 saved', 'GN7-13197 saved', 'GN7-13686 saved', 'GN7-13777 saved', 'GN7-13909 saved', 'GN7-14184 saved', 'GN7-14281 saved', 'GN7-14716 saved', 'GN7-14850 saved', 'GN2-14895 saved', 'GN7-15127 saved', 'GN7-15204 saved', 'GN7-15300 saved', 'GN7-15761 saved', 'GN7-16041 saved', 'GN2-16173 saved', 'GN2-16752 saved', 'GN7-17532 saved', 'GN2-17579 saved', 'GN2-17829 saved', 'GN7-17927 saved', 'GN2-18197 saved', 'GN2-18224 saved', 'GN2-18315 saved', 'GN7-19005 saved', 'GN4-19075 saved', 'GN7-19235 saved', 'GN7-19258 saved', 'GN7-19504 saved', 'GN7-19659 saved', 'GN2-21552 saved', 'GN4-21690 saved', 'GN2-21720 saved', 'GN4-22547 saved', 'GN4-22815 saved', 'GN4-23082 saved', 'GN7-23580 saved', 'GN4-23756 saved', 'GN4-24377 saved', 'GN4-24582 saved', 'GN4-26015 saved', 'GN4-27282 saved', 'GN3-28121 saved', 'GN4-28379 saved', 'GN3-30204 saved', 'GN5-31789